# A Guide to Time Series Forecasting with ARIMA

这个Notebook主要介绍了如何利用 ARIMA 基于现有的时间序列数据进行预测。


## 系统要求

1. Language: Python 3
2. Library:  pandas, statsmodels, matplotlib, ~~warnings, itertools~~, numpy
3. Others(Optional): Annaconda, Jupyter notebook

## Step 1 Loading Time-series Data

### 1）载入系统所需库

In [ ]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

### 2）工作的数据集

- 名称：Atmospheric CO2 from Continuous Air Samples at Mauna Loa Observatory, Hawaii, U.S.A.
- 内容：收集了美国从 1958 年 3 月到 2001 年 12 月期间采样的 CO2 数据。

In [ ]:
data = sm.datasets.co2.load_pandas()
y = data.data

- 转换**周**数据到**月**数据，并填充空值

In [ ]:
# The 'MS' string groups the data in buckets by start of the month
y = y['co2'].resample('MS').mean()

# The term bfill means that we use the value before filling in missing values
y = y.fillna(y.bfill())

print(y)

- 画图

In [ ]:
y.plot(figsize=(15, 6))
plt.show()

## Step 2 The ARIMA Time Series Model

### ARIMA = **A**utoreg**R**essive **I**ntegrated **M**oving **A**verage

描述 ARIMA 模型的三元组（p, d, q) 
- **p** 是模型的自回归部分。 它允许我们将过去值的影响纳入我们的模型。 直观地说，这类似于声明如果过去 3 天天气温暖，明天可能会温暖。
- **d** 是模型的集成部分。 这包括模型中包含差异量（即从当前值中减去的过去时间点的数量）以应用于时间序列的项。 直觉上，这类似于声明如果过去三天的温度差异非常小，明天的温度可能相同。
- **q** 是模型的移动平均部分。 这允许我们将模型的误差设置为过去之前时间点观察到的误差值的线性组合。

### Seasonal ARIMA: ARIMA(p,d,q)(P,D,Q)s
- (p, d, q) 是非季节性参数
- (P, D, Q) 是时间序列的季节性成分
- s 是时间序列的周期（4 个季度，12 个年度，等等）


## Step 3 Parameter Selection for the ARIMA Time Series Model

**目标**：选择合适的 (p,d,q), (P,D,Q) 和 s

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

## Step 4 — Fitting an ARIMA Time Series Model

- 显示特定参数模型下的结果
    - coef: 不同特征的权重值
    - p>|z|: 特征权重的显著度

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

- 图示

In [ ]:
results.plot_diagnostics(figsize=(15, 12))
plt.show()

## Step 5 — Validating Forecasts

如果模型的参数已经确定，就可以利用模型对数据进行预测。

相关的函数
    - get_prediction()
    - conf_int()

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('1998-01-01'), dynamic=False)
pred_ci = pred.conf_int()

- 利用 **前期的历史数据** 进行静态预测
    - dynamics = FALSE

In [ ]:
ax = y['1990':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')
plt.legend()

plt.show()

- 利用 MSE 来描述预测的平均误差

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = y['1998-01-01':]

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

- 利用 **“历史数据+预测数据”** 进行动态预测
    - dynamics = TRUE

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('1998-01-01'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

ax = y['1990':].plot(label='observed', figsize=(20, 15))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('1998-01-01'), y.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')

plt.legend()
plt.show()

- 利用 MSE 来描述预测的平均误差

In [ ]:
# Extract the predicted and true values of our time series
y_forecasted = pred_dynamic.predicted_mean
y_truth = y['1998-01-01':]

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

## Step 6 — Producing and Visualizing Forecasts

In [ ]:
# Get forecast 500 steps ahead in future
pred_uc = results.get_forecast(steps=500)

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()

In [ ]:
ax = y.plot(label='observed', figsize=(20, 15))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('CO2 Levels')

plt.legend()
plt.show()